# Minería de Opiniones con Keras

[Keras](https://keras.io/) es una librería de alto nivel que permite crear redes neuronales en forma muy sencilla.

Para trabajar con keras primero se tiene que instalar la librería [TensorFlow](https://www.tensorflow.org/install/):

pip3 install --upgrade tensorflow
pip install keras

# Tokenizar el corpus de texto

**Para este ejemplo, se asumirá que el corpus ya se encuentra pre-procesado y normalizado, sin olvidar y remarcar que esta tarea es muy importante.**

**Es importante contar con un corpus de texto balanceado y con muchas frases, ya que las redes neuronales a menudo requieren grandes cantidades de datos para obtener un buen entrenamiento y por lo tanto un buen "accuracy". Sin embargo, a mayor tamaño del corpus, se requiere más tiempo de entrenamiento.**

Antes de crear el modelo de red neuronal con Keras, es necesario separar cada texto/frase del corpus en palabras y representar cada palabra (token) por un número. 

Por lo tanto, se tiene que calcular la frecuencia de cada palabra (token) del corpus de texto, con el objetivo de encontrar las palabras más comunes (top) y asignarle una valor pequeño. 

Por ejemplo, Keras representa cada palabra (token) como un número, en dónde la palabra más común del corpus se representa con el número 1, la segunda palabra más común con el número 2, y así sucesivamente. 

Esto quiere decir que la numeración de las palabras (tokens) raras/inusuales/menos frecuentes tienen un valor muy alto. En la mayoría de los casos esas palabras no ayudan al entrenamiento de la red neuronal, por lo que no son de utilidad.

Por otro lado, si las palabras (top) más representativas tienen valores pequeños, entonces es más fácil entrenar nuestra red neuronal con esas N-palabras más representativas, y ajustar el valor de N en caso de que sea necesario. En keras, el atributo num_words se utiliza para tomar las n_palabras más frecuentes.

* num_words = es un atributo que toma en cuenta solo las n palabras mas frecuentes

Una vez que se cuenta con cada palabra (token), el siguiente pasao es transformar cada palabra a una representación númerica, con un ID entero que permitirá su representación vectorial.

La clase Tokenizer tiene varios métodos que ayudan a preparar el texto para que pueda usarse en el modelo de red neuronal. Entre los más importantes: 

* fit_on_text = construye el indice de palabras
* text_to_secuences = convierte un texto a un array númerico
* pad_sesequences = hace que todos los textos tengan la misma longitud (rellena con ceros)

**Nota: Al cálculo de las frecuencias de cada palabra se le conoce como fitting, y su representación númerica  se le llama secuencias.**

In [1]:
# -*- coding: utf-8 -*-
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# el corpus contiene seis documentos con sus respectivas etiquetas (+,-)
dataset = ['ME gusta este ejercicio :-)',
           'no prepara su clase!!', 
           'no me gusta su clase.', 
           'es algo dificil de, hacer',
           'tiene Mucho conocimiento',
           'me gusta mucho su clase']
etiquetas = ['positivo','negativo','negativo','negativo','positivo','positivo']

nb_words = 10 # toma en cuenta solo las 10 palabras más frecuentes

# crea el tokenizer con los las palabras mas frecuentes
tokenizer = Tokenizer(num_words = nb_words) 

# adapta (fit) el tokenizer a los documentos
# el tokenizer reemplaza las palabras con un ID entero para obtener
# su representación vectorial
tokenizer.fit_on_texts(dataset) 

Using TensorFlow backend.


In [2]:
# sumariza lo que ha aprendizo el tokenizer
print("Vocabulario aprendido:")
print(">>>",tokenizer.word_counts)
print(">>>",tokenizer.document_count)
print(">>>",tokenizer.word_index)
print(">>>",tokenizer.word_docs)
print("Total palabras vocabulario:",len(tokenizer.word_index)+1)

Vocabulario aprendido:
>>> {'no': 2, 'me': 3, 'tiene': 1, 'dificil': 1, 'su': 3, 'ejercicio': 1, 'es': 1, 'este': 1, 'de': 1, 'algo': 1, 'gusta': 3, 'hacer': 1, 'clase': 3, 'conocimiento': 1, 'prepara': 1, 'mucho': 2}
>>> 6
>>> {'no': 5, 'me': 1, 'prepara': 16, 'dificil': 8, 'su': 2, 'tiene': 7, 'es': 10, 'este': 11, 'de': 12, 'algo': 13, 'gusta': 3, 'hacer': 14, 'clase': 4, 'conocimiento': 15, 'ejercicio': 9, 'mucho': 6}
>>> {'no': 2, 'me': 3, 'tiene': 1, 'dificil': 1, 'su': 3, 'ejercicio': 1, 'es': 1, 'este': 1, 'de': 1, 'algo': 1, 'gusta': 3, 'hacer': 1, 'clase': 3, 'conocimiento': 1, 'prepara': 1, 'mucho': 2}
Total palabras vocabulario: 17


Observa la salida anterior. La clase Tokenizer realiza un filtrado básico de los signos de puntuación. También es posible crear expresiones regulares para realizar un filtrado más complejo.

Observa que el método texts_to_sequences convierte un texto a un array númerico (lista de índices enteros) pero tomando como base el vocabulario del tokenizador, por lo que la palabra complicado no se representará en el array.

In [3]:
tokenizer.texts_to_sequences(['me gusta este ejercicio complicado'])

[[1, 3, 9]]

In [4]:
tokenizer.texts_to_sequences(dataset)

[[1, 3, 9], [5, 2, 4], [5, 1, 3, 2, 4], [8], [7, 6], [1, 3, 6, 2, 4]]

Si queremos alimentar a la red con nuevas oraciones, se utiliza el método text_to_matrix que convierte cada frase el arreglos de igual tamaño.

In [5]:
tokenizer.texts_to_matrix(['hola mundo java'])

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

Keras cuenta con la función pad_sequences, que rellenará con ceros los espacios faltantes en cada texto para que todos los textos tengan la misma longitud.

In [6]:
# las secuencias de texto no tienen la misma longitus
text_sec = tokenizer.texts_to_sequences(dataset)
print(text_sec)

[[1, 3, 9], [5, 2, 4], [5, 1, 3, 2, 4], [8], [7, 6], [1, 3, 6, 2, 4]]


In [7]:
# rellenamos los huecos con ceros para que las frases tengan la misma longitud
padded_text = pad_sequences(text_sec)
print(padded_text)

[[0 0 1 3 9]
 [0 0 5 2 4]
 [5 1 3 2 4]
 [0 0 0 0 8]
 [0 0 0 7 6]
 [1 3 6 2 4]]


In [28]:
#Ejemplo para jugar con num_words = 4, 8, 12 y max_pad = 5 y 10
nb_words = 4
max_pad = 5 
tokenizer = Tokenizer(num_words = nb_words)
tokenizer.fit_on_texts(dataset)
secuences = tokenizer.texts_to_sequences(dataset)
data = pad_sequences(secuences, maxlen = max_pad)  
print(data)

[[0 0 0 1 2]
 [0 0 0 0 3]
 [0 0 1 2 3]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 2 3]]


# Configuración del modelo

1. Crear un modelo secuencial vacio: Sequential()
2. Al modelo vacío se le agregan una o más capas dependiendo de la arquitectura de red neuronal que se quiere modelar
3. Se tiene que agregar una capa Embedding (vocabulario, vector, longitud de la secuencia)
   La capa de salida será de 32x5

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten

model = Sequential()
model.add(Embedding(10, 32, input_length=5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 1. Cargar y procesar el dataset

In [9]:
from collections import Counter
from datetime import datetime
import json
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation, Flatten, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from numpy.random import seed
from tensorflow import set_random_seed
import keras
from keras.callbacks import History, ModelCheckpoint

# seed for reproducing same results
seed(1)
set_random_seed(2)

In [10]:
# Load the reviews and parse JSON
t1 = datetime.now()
with open("yelp_academic_dataset_review.json") as f:
    reviews = f.read().strip().split("\n")
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

0:00:05.992660


In [11]:
# Get a balanced sample of positive and negative reviews
texts = [review['text'] for review in reviews]

# Convert our 5 classes into 2 (negative or positive)
binstars = [0 if review['stars'] <= 3 else 1 for review in reviews]
balanced_texts = []
balanced_labels = []
limit = 73836 # Change this to grow/shrink the dataset
neg_pos_counts = [0, 0]
for i in range(len(texts)):
    polarity = binstars[i]
    if neg_pos_counts[polarity] < limit:
        balanced_texts.append(texts[i])
        balanced_labels.append(binstars[i])
        neg_pos_counts[polarity] += 1

In [12]:
Counter(balanced_labels)
# >>> Counter({0: 100000, 1: 100000})

Counter({0: 73836, 1: 73836})

# 2. Obtener tokens y word embedding

In [ ]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(balanced_texts)
sequences = tokenizer.texts_to_sequences(balanced_texts)
data = pad_sequences(sequences, maxlen=300)

# Modelo 1: Multilayer Perceptron, vocab = 20000, batch_size = 32 

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)
model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

Train on 98940 samples, validate on 48732 samples
Epoch 1/100
32896/98940 [========>.....................] - ETA: 1295s - loss: 0.4279 - acc: 0.8045

# Modelo 2: CNN, Vocab = 20000, batch_size = 32

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Dropout(0.5))
model.add(Flatten())    
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 3: CNN Variación en los filtros y kernel, vocab = 20000, batch_size = 32

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.5))
model.add(Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation='relu'))
model.add(MaxPooling1D(pool_size= 2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())    
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 4: CNN con GlobalMaxPooling, vocab = 20000, batch_size = 32

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Conv1D(filters=250, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 5: RNN-LSTM, vocab = 20000, batch_size = 32

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)
model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 6: CNN-LSTM, vocab = 32, batch_size = 32

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 7: CNN con multiples capas y neuronas, vocab = 20000, batch_size = 32

In [ ]:
import keras
from keras.callbacks import History, ModelCheckpoint

model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.2))
model.add(Conv1D(300, 3, padding = 'valid', activation='relu', strides = 2))
model.add(Conv1D(150, 3, padding = 'valid', activation='relu', strides = 2))
model.add(Conv1D(75, 3, padding = 'valid', activation='relu', strides = 2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(150, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 8: LSTM con otra versión, vocab = 20000, batch_size = 32

In [ ]:
from keras import regularizers
model = Sequential()
model.add(Embedding(20000, 128,input_length = 300))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model.add(Dense(1,activation='softmax', kernel_regularizer=regularizers.l2(0.001),
                activity_regularizer=regularizers.l1(0.001)))
model.compile(loss = 'binary_crossentropy', optimizer='adadelta',metrics = ['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 9: CNN-LSTM, vocab = 20000, batch_size = 32

In [ ]:
model = Sequential()
    #model.add(Embedding(max_features, EMB_VECTOR_LENGHT, input_length=MAX_SEQUENCE_LENGHT))
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.5))
model.add(Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation='relu'))
model.add(MaxPooling1D(pool_size= 2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=6, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(LSTM(256))
model.add(Dropout(0.5))
    #model.add(Flatten())
    #128,256,512
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 10: CNN, vocabulario = 20000, batch_size = 32

In [ ]:
model = Sequential()
    #model.add(Embedding(max_features, EMB_VECTOR_LENGHT, input_length=MAX_SEQUENCE_LENGHT))
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.5))
model.add(Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation='relu'))
model.add(MaxPooling1D(pool_size= 2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=6, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
#model.add(LSTM(256))
#model.add(Dropout(0.5))
model.add(Flatten())
    #128,256,512
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))

# Modelo 10: CNN, vocabulario = 20000, batch_size = 64

In [ ]:
model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(Dropout(0.5))
model.add(Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation='relu'))
model.add(MaxPooling1D(pool_size= 2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=32, kernel_size=6, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='max')
mcp = ModelCheckpoint("corpus.hdf5", monitor="val_acc", save_best_only=True, save_weights_only=False,verbose = 1)

model.fit(data, np.array(balanced_labels), validation_split=0.33, epochs=100, 
                          callbacks=[earlyStopping,mcp], verbose=1, batch_size= 64)

# Evalua el modelo
scores = model.evaluate(data, np.array(balanced_labels), verbose=0)
print("Evaluation result on Test Data.")
print("Loss: %.2f%%" % scores[0])
print("Accuracy: %.2f%%" % (scores[1] * 100))